In [10]:
## Dataset should be arranged in two folder,
##### Dataroot
#####  |
#####  ├── images (jpeg/png RGB images)
#####  └── labels (label files in yolo format. Each label file should have same basename as corresponding image file)

##### The code must run from project root directory (For Example: ~/work/yolov5)

> Run the notebook one section at a time only. 
>
> WARNING: Running the entire notebook at once will cause errors !!!

## 1. Training

In [5]:
# dataset parameters
dataRoot = "/home/varun/work/yolov5/data/labData/corn-n-soy"
trainRelativePath = "images"
valRelativePath = "images"
yamlPath = "/home/varun/work/yolov5/data/labData/corn-n-soy.yaml"
classList = ["weed"]

# training parameters
imgSize = 640
batchSize = 32
epochs = 300
weightPath = "/home/varun/work/yolov5/yolov5s.pt"
device = "0"
optimizer = "Adam" #(choose from 'SGD', 'Adam', 'AdamW')  
workers = 16
projectSaveDir = "/home/varun/work/yolov5/runs/acres22young/train"
projectName = "corn-n-soy-v1" # name of the project (relative to projectSaveDir)
savePeriod = 10
cacheRAM = True
useWandb = True
condaEnvName = "yv5"

In [6]:
import os

pN = os.path.join(projectSaveDir, projectName)
## Setups train command
if cacheRAM:
    trainString = f"python train.py --img {imgSize} --batch {batchSize} --epochs {epochs} --data {yamlPath} --weights {weightPath} --device {device} --optimizer {optimizer} --workers {workers} --project {pN} --save-period {savePeriod} --cache ram"
else:
    trainString = f"python train.py --img {imgSize} --batch {batchSize} --epochs {epochs} --data {yamlPath} --weights {weightPath} --device {device} --optimizer {optimizer} --workers {workers} --project {pN} --save-period {savePeriod}"

In [7]:
## Create yaml file
with open(yamlPath, "w") as f:
    f.write("# YAML file for Project: "+ projectName+"\n")
    f.write("# Training Command:\n")
    f.write("# "+trainString+ "\n")
    f.write("train: " + os.path.join(dataRoot, trainRelativePath) + "\n")
    f.write("val: " + os.path.join(dataRoot, valRelativePath) + "\n")
    f.write("nc: " + str(len(classList)) + "\n")
    f.write("names:\n")
    for i, cls in enumerate(classList):
        f.write("  "+str(i)+": "+cls + "\n")

In [8]:
# create train run script
with open("runTrain_"+projectName+".sh", "w") as f:
    f.write("#!/bin/bash\n")
    f.write(trainString)

print("Run the following commands to start training:")
print("tmux new -s "+projectName)
print("ca yv5")
if useWandb:
    print("wandb online")
else:
    print("wandb offline")
print("sh runTrain_"+projectName+".sh")

Run the following commands to start training:
tmux new -s corn-n-soy-v1
ca yv5
wandb online
sh run_corn-n-soy-v1.sh


# 2. Testing

In [28]:
# input related parameters
projectName = "corn-n-soy-v1" # for reading the weights
exp = "exp"
weightsDir = f"runs/acres22young/train/{projectName}/{exp}/weights/best.pt"

# output related parameters
testCrop = 'soy' # 'corn' or 'soy'
day = 'day0' # day0, day3, day5
testImgsDir = f"split/{testCrop}/{day}/*"
projectRootDir = "/home/varun/work/yolov5/runs/acres22young/detect/"
detectSaveName = f"{projectName}-{exp}-{testCrop}-{day}" # projectName for saving the detection results (Name is mix of training dataset and test dataset)

# detection parameters
confThres = 0.25
iouThres = 0.45

In [29]:
# create detect run script
testString = f"python detect.py --weights {weightsDir} --source '{testImgsDir}' --save-txt --project {projectRootDir} --name {detectSaveName} --conf-thres {confThres} --iou-thres {iouThres}"
with open("runDetect_"+detectSaveName+".sh", "w") as f:
    f.write("#!/bin/bash\n")
    f.write(testString)

print("Ensure you in root directory of yolov5, then run the following commands to start training:")
print()
print("tmux new -s "+detectSaveName)
print("ca yv5")
print("sh runDetect_"+detectSaveName+".sh")

Ensure you in root directory of yolov5, then run the following commands to start training:

tmux new -s corn-n-soy-v1-exp-soy-day0
ca yv5
sh runDetect_corn-n-soy-v1-exp-soy-day0.sh


# 3. Counting and validating with ground truth task

This is part of research effort in collboration with Dr. Young in 2022.
Dataset: images collected at ACRES in Summer of 2022

In [ ]:
#Files to read
rootDataFolder = 'runs/acres22young/dataSideProject'
cornCountYoung = 'ACRE-069-CORN_Weed Counts and Application Info.xlsx'
cornCountVarun = 'results.xlsx'
sheetCornCountYoung = 'Total Weed Density by Specie'
sheetCornCountVarun = 'Corn - Day0'

soyCountYoung = 'ACRE-131-SOYBEAN_Weed Counts and Application Info.xlsx'
soyCountVarun = 'results.xlsx'
sheetSoyCountYoung = 'Total Weed Density by Specie'
sheetSoyCountVarun = 'Soy - Day0'

#Folder in which output should be saved
save_folder = 'SP_Reports'

In [ ]:
import os
import re
import collections
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

# Handling File Paths
cornCountYoung = os.path.join(rootDataFolder, cornCountYoung)
cornCountVarun = os.path.join(rootDataFolder, cornCountVarun)
soyCountYoung = os.path.join(rootDataFolder, soyCountYoung)
soyCountVarun = os.path.join(rootDataFolder, soyCountVarun)

#Creating save_folder
if save_folder not in os.listdir():
    os.mkdir(save_folder)

### 3.2 Corn Processing

In [ ]:
dfYoung = pd.read_excel(cornCountYoung, sheet_name=sheetCornCountYoung)
dfVarun = pd.read_excel(cornCountVarun, sheet_name=sheetCornCountVarun)

# Merge the two dataframes for corn
plotCountYoung = dfYoung.query('DAA==0')[['TOTAL','Plot','Corn Stage']].sort_values(by='Plot').reset_index(drop=True)
plotCountVarun = dfVarun.copy()
dfTemp = pd.merge(plotCountYoung, plotCountVarun, left_on='Plot', right_on='Plot Number', how='inner')
dfTemp = dfTemp[['Plot','Corn Stage','TOTAL','Weed Count']]
dfTemp.rename(columns={'TOTAL':'Young Count', 'Weed Count':'Varun Count', 'Plot':'Plot Corn'}, inplace=True)

# Get difference of weed count for corn
dfTemp['Difference'] = dfTemp['Young Count'] - dfTemp['Varun Count']

dfTemp.to_excel(os.path.join(save_folder, 'Corn_Compiled.xlsx'), index=False)
display(dfTemp.head())

### 3.3 Soy Processing

In [ ]:
# Soy Processing
dfYoung = pd.read_excel(soyCountYoung, sheet_name=sheetSoyCountYoung)
dfVarun = pd.read_excel(soyCountVarun, sheet_name=sheetSoyCountVarun)

# Merge the two dataframes for soy
plotCountYoung = dfYoung.query('DAA==0')[['TOTAL','Plot','Soybean Stage']].sort_values(by='Plot').reset_index(drop=True)
plotCountVarun = dfVarun.copy()
dfTemp = pd.merge(plotCountYoung, plotCountVarun, left_on='Plot', right_on='Plot Number', how='inner')
dfTemp = dfTemp[['Plot','Soybean Stage','TOTAL','Weed Count']]
dfTemp.rename(columns={'TOTAL':'Young Count', 'Weed Count':'Varun Count', 'Plot':'Plot Soy'}, inplace=True)

# Get difference of weed count for soy
dfTemp['Difference'] = dfTemp['Young Count'] - dfTemp['Varun Count']

dfTemp.to_excel(os.path.join(save_folder, 'Soy_Compiled.xlsx'), index=False)
display(dfTemp.head())